In [2]:

from flask import Flask, request, jsonify, Response
from threading import Thread
import json
import datetime
import pytz
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.exceptions import RefreshError
from openai import OpenAI
import re

# --- Part 2: Flask Server Setup ---
app = Flask(__name__)
app.config['JSON_SORT_KEYS'] = False # Force Flask to respect dictionary order

# --- Part 3: Our AI and Calendar Logic ---

client = OpenAI(base_url="http://0.0.0.0:3000/v1", api_key="vllm")
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"

class AI_AGENT:
    def parse_email(self, email_text, attendees_list):
        attendee_str = ", ".join(attendees_list)
        response = client.chat.completions.create(
            model=MODEL_PATH, temperature=0.0,
            messages=[{
                "role": "user",
                "content": f"""
                You are a precision AI data extraction service.
                Analyze the **Email Content** and **List of Attendees** to extract exactly four fields.
                - **Email Content**: "{email_text}"
                - **List of Attendees**: "{attendee_str}"
                1.  **participants**: Use emails from the `List of Attendees`.
                2.  **time_constraints**: Extract the DAY or RELATIVE DATE ONLY (e.g., "Monday").
                3.  **meeting_duration**: Calculate total minutes as an INTEGER (e.g., 90).
                4.  **meeting_time**: Extract the CLOCK TIME ONLY (e.g., "9:00 AM") or return null.
                Your entire response MUST be a single, valid JSON object containing ONLY these four keys.
                """
            }]
        )
        return json.loads(response.choices[0].message.content)

    def judger(self, email_text, llm_output):
        # For the final version, we will trust the first parser to be fast and accurate.
        return llm_output

# Configuration: Mapping
ATTENDEE_TO_TOKEN_USER_MAPPING = {
    'userone.amd@gmail.com': 'userone.amd@gmail.com',
    'usertwo.amd@gmail.com': 'usertwo.amd@gmail.com',
    'userthree.amd@gmail.com': 'userthree.amd@gmail.com',
}

def retrive_calendar_events(user, start, end):
    token_path = "../Keys/" + user.split("@")[0] + ".token"
    try:
        creds = Credentials.from_authorized_user_file(token_path)
        service = build("calendar", "v3", credentials=creds)
        events_result = service.events().list(calendarId='primary', timeMin=start, timeMax=end, singleEvents=True, orderBy='startTime').execute()
        
        formatted_events = []
        for event in events_result.get('items', []):
            if 'dateTime' not in event['start']:
                continue
            attendees = [att.get('email') for att in event.get('attendees', []) if att.get('email')]
            if not attendees: attendees.append("SELF")
            formatted_events.append({
                "StartTime": event['start']['dateTime'],
                "EndTime": event['end']['dateTime'],
                "NumAttendees": len(attendees),
                "Attendees": attendees,
                "Summary": event.get('summary', 'No Title')
            })
        return formatted_events
    except (RefreshError, FileNotFoundError):
        return []

def find_meeting_slot(attendees, time_constraint, duration_mins, requested_time_str):
    all_users_calendars = {}
    today = datetime.datetime.now(pytz.timezone('Asia/Kolkata'))
    target_date = None
    time_constraint_lower = time_constraint.lower()
    
    weeks_match = re.search(r'(\d+)\s+weeks?', time_constraint_lower)
    days_match = re.search(r'(\d+)\s+days?', time_constraint_lower)
    
    if 'tomorrow' in time_constraint_lower: target_date = today + datetime.timedelta(days=1)
    elif weeks_match: target_date = today + datetime.timedelta(weeks=int(weeks_match.group(1)))
    elif days_match: target_date = today + datetime.timedelta(days=int(days_match.group(1)))
    else:
        days_map = {'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3, 'friday': 4}
        target_day = next((day_index for day_name, day_index in days_map.items() if day_name in time_constraint_lower), -1)
        if target_day != -1:
            days_ahead = (target_day - today.weekday() + 7) % 7
            target_date = today + datetime.timedelta(days=days_ahead)
        else: target_date = today + datetime.timedelta(days=1)

    if target_date.weekday() >= 5: target_date += datetime.timedelta(days=(7 - target_date.weekday()))

    day_start = target_date.replace(hour=9, minute=0, second=0, microsecond=0)
    day_end = target_date.replace(hour=17, minute=0, second=0, microsecond=0)

    all_busy_slots = []
    for email in attendees:
        token_user = ATTENDEE_TO_TOKEN_USER_MAPPING.get(email)
        if not token_user: continue
        user_events = retrive_calendar_events(token_user, day_start.isoformat(), day_end.isoformat())
        all_users_calendars[email] = user_events
        for event in user_events:
            all_busy_slots.append({"start": datetime.datetime.fromisoformat(event['StartTime']), "end": datetime.datetime.fromisoformat(event['EndTime'])})
    
    all_busy_slots.sort(key=lambda x: x["start"])
    merged_busy_slots = [all_busy_slots[0]] if all_busy_slots else []
    for current_slot in all_busy_slots[1:]:
        last_slot = merged_busy_slots[-1]
        if current_slot["start"] < last_slot["end"]: last_slot["end"] = max(last_slot["end"], current_slot["end"])
        else: merged_busy_slots.append(current_slot)
    
    meeting_duration = datetime.timedelta(minutes=duration_mins)
    search_start_time = day_start
    if requested_time_str:
        try:
            parsed_time = datetime.datetime.strptime(requested_time_str.upper().replace('.', ''), "%I:%M %p").time()
            requested_start = day_start.replace(hour=parsed_time.hour, minute=parsed_time.minute)
            requested_end = requested_start + meeting_duration
            is_free = not any(max(requested_start, slot["start"]) < min(requested_end, slot["end"]) for slot in merged_busy_slots)
            if is_free and day_start <= requested_start and requested_end <= day_end:
                return {"EventStart": requested_start.isoformat(), "EventEnd": requested_end.isoformat()}, all_users_calendars
            else:
                conflicting_slot = next((slot for slot in merged_busy_slots if max(requested_start, slot["start"]) < min(requested_end, slot["end"])), None)
                if conflicting_slot: search_start_time = max(search_start_time, conflicting_slot["end"])
        except ValueError: pass 
    
    last_busy_end = search_start_time
    for slot in merged_busy_slots:
        if slot["start"] > last_busy_end:
            if slot["start"] - last_busy_end >= meeting_duration:
                return {"EventStart": last_busy_end.isoformat(), "EventEnd": (last_busy_end + meeting_duration).isoformat()}, all_users_calendars
        last_busy_end = max(last_busy_end, slot["end"])
    if day_end - last_busy_end >= meeting_duration:
        return {"EventStart": last_busy_end.isoformat(), "EventEnd": (last_busy_end + meeting_duration).isoformat()}, all_users_calendars
    return None, all_users_calendars

def your_meeting_assistant(input_json):
    agent = AI_AGENT()
    email_content = input_json.get("EmailContent", "")
    attendees_from_input = [att["email"] for att in input_json.get("Attendees", [])]
    organizer = input_json.get("From")
    if organizer and organizer not in attendees_from_input:
        attendees_from_input.append(organizer)

    ai_output = agent.parse_email(email_content, attendees_from_input)

    try: duration = int(ai_output.get('meeting_duration', 30))
    except (ValueError, TypeError): duration = 30
    
    time_constraint = ai_output.get('time_constraints', 'tomorrow')
    requested_time = ai_output.get('meeting_time')
    
    found_slot, all_calendars = find_meeting_slot(attendees_from_input, time_constraint, duration, requested_time)
    
    start_time, end_time = (found_slot.get("EventStart"), found_slot.get("EventEnd")) if found_slot else (None, None)

    # FINAL FIX: Build the output dictionary in the exact required order
    final_output = {
        "Request_id": input_json.get("Request_id"),
        "Datetime": input_json.get("Datetime"),
        "Location": input_json.get("Location"),
        "From": input_json.get("From"),
        "Attendees": [{"email": email, "events": all_calendars.get(email, [])} for email in attendees_from_input],
        "Subject": input_json.get("Subject"),
        "EmailContent": input_json.get("EmailContent"),
        "EventStart": start_time,
        "EventEnd": end_time,
        "Duration_mins": str(duration),
        "MetaData": {}
    }
    
    return final_output


@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    final_dict = your_meeting_assistant(data)
    
    
    final_json_string = json.dumps(final_dict, indent=2, sort_keys=False)
    
    
    return Response(final_json_string, mimetype='application/json')

def run_flask():
    app.run(host='0.0.0.0', port=5000)


Thread(target=run_flask, daemon=True).start()


✅ Final, correctly ordered server started on port 5000.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.191.96:5000
Press CTRL+C to quit
129.212.191.96 - - [20/Jul/2025 08:30:29] "POST /receive HTTP/1.1" 200 -
129.212.191.96 - - [20/Jul/2025 08:30:42] "POST /receive HTTP/1.1" 200 -
129.212.191.96 - - [20/Jul/2025 08:31:55] "POST /receive HTTP/1.1" 200 -
